<font size="6" face="Times"> <b>Create Categories File</b> </font>

<b>Variables</b>

In [1]:
read_raw_data = "D:/Unisinos/Bitalino/01 - Raw Data/All Data/"
read_annotations = "D:/Unisinos/Bitalino/02 - Data Control/annotations.csv"
write_data = "D:/Unisinos/Bitalino/03 - Data CSV/"
file_type = "_file.csv"

<b>Libraries</b>

In [2]:
import pickle
import pytz
import datetime
import itertools
import pandas as pd
import numpy as np
from os import listdir
from collections import Counter
from os.path import isfile, join

<b>Read All the Files and Process the Data </b>

In [3]:
_pwd = read_raw_data

In [4]:
files = {}
data = []
for participant_folder in listdir(_pwd):
    _participant = participant_folder.split(' ')[1]
    for folder in listdir(_pwd + participant_folder):
        print(folder)
        data_timestamp = int(folder.split('-')[1].lstrip())
        data_timestamp = datetime.datetime.fromtimestamp(data_timestamp)

        files = [f for f in listdir(_pwd+participant_folder+'/'+folder) if isfile(join(_pwd+participant_folder+'/'+folder, f))]

        for f in files:
            file_index = int(f.split('.')[0])
            data_values = pickle.load(open(_pwd+participant_folder+'/'+folder+'/'+f, "rb")).tolist()
            data_values = pd.DataFrame(data_values)
            data_values.columns = ['DIGITAL{0}'.format(x) for x in range(0,len(data_values.columns))]
            data_values.rename(columns={'DIGITAL6':'ECG','DIGITAL7':'EMG','DIGITAL8':'EDA'}, inplace=True)
            data_values['PARTICIPANT'] = _participant

            data_values['ECG'] = [((((x/1024)-(1/2))*3.3)/1100)*1000 for x in data_values['ECG']]

            data_values['FILEINDEX'] = file_index

            # At each entry counts as a more one milesecond (@1000Hz)
            row_objects = [1000]*len(data_values)
            row_objects = list(np.cumsum(row_objects))
            row_objects = [data_timestamp + datetime.timedelta(milliseconds=int(x)) for x in row_objects]

            data.append(pd.concat([pd.DataFrame(row_objects, columns=['TIMESTAMP']), data_values], axis=1))

201806130369 - 1564678273
201806130369 - 1564678784
201806130369 - 1573754008
201806130369 - 1573754318
201806130369 - 1573755364
201806130369 - 1573757623
201806130369 - 1573757727
201806130369 - 1571157902
201806130369 - 1571158125
201806130369 - 1571159209
201806130369 - 1571159603
201806130369 - 1571159660
201806130369 - 1561651609
201806130369 - 1561651799
201806130369 - 1561651880
201806130369 - 1561652059
201806130369 - 1561652432
201806130369 - 1561652546
201806130369 - 1561652596
201806130369 - 1561653093
201806130369 - 1561653144
201806130369 - 1561653283
201806130369 - 1561654308
201806130369 - 1561654366
201806130369 - 1561654439
201806130369 - 1561654481
201806130369 - 1561654554
201806130369 - 1567097819
201806130369 - 1567098307
201806130369 - 1567098361
201806130369 - 1567098679
201806130369 - 1567099364


<b>Merge All Data into a Same Data Frame </b>

In [5]:
data = pd.concat(data)

Apply the TIMESTAMP

In [6]:
data['TIMESTAMP'] = data['TIMESTAMP'].apply(lambda x: x.to_pydatetime().replace(tzinfo=pytz.timezone('America/Sao_Paulo')))

In [7]:
data = data.sort_values(['TIMESTAMP'])

Delete unused columns

In [8]:
data=data.drop(['DIGITAL0','DIGITAL1','DIGITAL2','DIGITAL3','DIGITAL4','DIGITAL5','DIGITAL9','DIGITAL10','FILEINDEX'], axis=1)

<b>Read the Annotations and Preprocess<b>

Read the File

In [16]:
annotations = pd.read_csv(read_annotations, sep=';', encoding='utf-8')

Replace NaN with None

In [17]:
annotations = annotations.where((pd.notnull(annotations)), None)

In [18]:
annotations['PARTICIPANTE'] = annotations['PARTICIPANTE'].apply(lambda x: x.lstrip().rstrip() if x is not None else x)


Method to Combine <b>Date</b> and <b>Time</b> into a <b>Datetime</b>

In [19]:
def combine_date_time(xdate, xtime):
    new_feature = []
    for d,t in zip(xdate,xtime):
        if d is None or t is None:
            new_feature.append(None)
        else:
            new_feature.append(datetime.datetime.strptime(d + ' ' + t, '%d/%m/%Y %H:%M'))
    return new_feature

In [20]:
def combine_date_time(xdate, xtime):
    new_feature = []
    for d,t in zip(xdate,xtime):
        if d is None or t is None:
            new_feature.append(None)
        else:
            new_feature.append(datetime.datetime.strptime(t, '%d-%m-%Y %H:%M'))
    return new_feature

Cast <b>Start Time</b> and <b>End Time</b> into <b>Datetime</b>

In [21]:
annotations['HORAINICIO'] = combine_date_time(annotations['DATA'],annotations['HORAINICIO'])
annotations['HORAFINAL'] = combine_date_time(annotations['DATA'],annotations['HORAFINAL'])

Cast <b>timer</b> into <b>datetime.time</b>

In [22]:
annotations['CRONOMETROINICIO'] = annotations['CRONOMETROINICIO'].apply(lambda x: datetime.datetime.strptime(x, '%H:%M:%S').time() if isinstance(x, str) else None)
annotations['CRONOMETROFINAL'] = annotations['CRONOMETROFINAL'].apply(lambda x: datetime.datetime.strptime(x, '%H:%M:%S').time() if isinstance(x, str) else None)

Fulfill Missing <b>HORAFINAL</b> Information

In [23]:
import itertools

def get_next(some_iterable, window=1):
    items, nexts = itertools.tee(some_iterable, 2)
    nexts = itertools.islice(nexts, window, None)
    return zip(items, nexts)

new_feature = []

for _current, _next in get_next(zip(annotations['PARTICIPANTE'], annotations['HORAINICIO'],annotations['HORAFINAL'])):
    if _next[0] == _current[0]:
        if (_current[1] is None) or (isinstance(_current[1],pd._libs.tslibs.nattype.NaTType)):
            new_feature.append(_current[1])
        else:
            if (_current[2] is None) or (isinstance(_current[2],pd._libs.tslibs.nattype.NaTType)):
                new_feature.append(_next[1]) # Add the next HORAINICIO value
            else:
                new_feature.append(_current[2]) # Add the current HORAFINAL value
    else:
        new_feature.append(_current[2]) # Add the current HORAFINAL value

new_feature.append(_current[2]) # Add the current HORAFINAL value

In [24]:
annotations['HORAFINAL'] = new_feature

Fulfill Missing <b>CRONOMETROFINAL</b> Information

In [25]:
import itertools

def get_next(some_iterable, window=1):
    items, nexts = itertools.tee(some_iterable, 2)
    nexts = itertools.islice(nexts, window, None)
    return zip(items, nexts)

new_feature = []

for _current, _next in get_next(zip(annotations['PARTICIPANTE'], annotations['CRONOMETROINICIO'],annotations['CRONOMETROFINAL'])):
    if _next[0] == _current[0]:
        if (_current[1] is None) or (isinstance(_current[1],pd._libs.tslibs.nattype.NaTType)):
            new_feature.append(_current[1])
        else:
            if (_current[2] is None) or (isinstance(_current[2],pd._libs.tslibs.nattype.NaTType)):
                new_feature.append(_next[1]) # Add the next HORAINICIO value
            else:
                new_feature.append(_current[2]) # Add the current HORAFINAL value
    else:
        new_feature.append(_current[2]) # Add the current HORAFINAL value

new_feature.append(_current[2]) # Add the current HORAFINAL value

In [26]:
annotations['CRONOMETROFINAL'] = new_feature

Create a New Feature from the <b>start time difference</b> and <b>end time difference</b>

In [27]:
new_feature = []
for start, end in zip(annotations['HORAINICIO'],annotations['HORAFINAL']):   
    if start is None or isinstance(start,pd._libs.tslibs.nattype.NaTType) or end is None or isinstance(end,pd._libs.tslibs.nattype.NaTType):
        new_feature.append(None)
    else:
        start = start.time()
        end = end.time()
        new_feature.append(datetime.datetime.combine(datetime.date.today(), end) - datetime.datetime.combine(datetime.date.today(), start))

In [28]:
annotations['HORADIFERENCA'] = new_feature

Create a New Feature from the <b>start timer difference</b> and <b>end timer difference</b>

In [29]:
new_feature = []
for start, end in zip(annotations['CRONOMETROINICIO'],annotations['CRONOMETROFINAL']):
    if start is None or isinstance(start,pd._libs.tslibs.nattype.NaTType) or end is None or isinstance(end,pd._libs.tslibs.nattype.NaTType):
        new_feature.append(None)
    else:
        new_feature.append(datetime.datetime.combine(datetime.date.today(), end) - datetime.datetime.combine(datetime.date.today(), start))

In [30]:
annotations['CRONOMETRODIFERENCA'] = new_feature

Cast the Timestamp type to datetime

In [31]:
annotations['HORAINICIO'] = annotations['HORAINICIO'].apply(lambda x: x.to_pydatetime())
annotations['HORAFINAL'] = annotations['HORAFINAL'].apply(lambda x: x.to_pydatetime())

Unify the many Annotations into Specified Categories

In [32]:
unifyAnnotation = {
    'baseline':[
        'Linha de base sensores (5min)',
        'Linha de base sensores'
    ],
    
    'tsst':[
        'Fala livre participante',
        'Instruções fala livre'
    ],
    
    'arithmetic':[
        'Instruções aritmética',
        'Tarefa de aritmética'
    ],
    
    'post_test_sensors_1':[
        'Pós-teste sensores 1'
    ],
    
    'post_test_sensors_2':[
        'Pós-teste sensores 2'
        
    ],
    
    'no_category':[
        'None',
        'Ligou Polar',
        'Ligou Esense',
        'Ligou Bewell',
        'Ligou Bewell 1 – mão',
        'Ligou Bewell 2 – peito',
        'Instruções para coleta da linha de base',
        'Pré-teste instrumentos',
        'Trajeto sala TSST',
        'Instruções para o participante',
        'Saída da sala pesquisadora',
        'Trajeto sala Pós-teste',
        'Saída da sala participante',
        'Pós-teste Instrumentos',
        'Trajeto sala Pós-teste',
        'Coleta Saliva',
        'Coleta Saliva I (-1min)',
        'Coleta Saliva II (+1min)',
        'Coleta Saliva III',
        'Coleta Saliva IV (+30min)',
        'Final – participante liberado',
        'Final - participante liberado',
        'Fim desligar sensores',
        'Fim - desligar sensores',
        'Preparação participante',
        'Fim – desligar sensores'
    ]

}

In [33]:
def mergeConfusingAnnotations(x):
    if x is None:
        return x
    
    x = x.lower().rstrip().lstrip()
    for k in unifyAnnotation.keys():
        if (k.lower() in x) or (x in k.lower()):
            return k
        else:
            for y in unifyAnnotation[k]:
                #print('{0} -> {1} : {2}'.format(x, y.lower(), (y.lower() in x) or (x in y.lower())))
                if (y.lower() in x) or (x in y.lower()):
                    return k
    return None

In [34]:
annotations['CATEGORIACOMPARACAO'] = annotations['CATEGORIA'].apply(lambda x: mergeConfusingAnnotations(x))

In [35]:
b = annotations[['CATEGORIA','CATEGORIACOMPARACAO']]

In [36]:
annotations['CATEGORIA'] = annotations['CATEGORIA'].apply(lambda x: mergeConfusingAnnotations(x))

<b>Relates the Data and Annotations</b>

Relates the Annotation <b>category</b> with Data by <b>participant</b>, <b>start time</b>, and <b>end time</b>

In [37]:
data['CATEGORY'] = None

In [38]:
_categories = {}

for pat in annotations['PARTICIPANTE'].unique():
    _participant = {}
    for cat in annotations['CATEGORIA'].unique():      
        condition = ((annotations['PARTICIPANTE'] == pat) & (annotations['CATEGORIA'] == cat))
        pat_min = annotations[condition]['HORAINICIO'].min()
        pat_max = annotations[condition]['HORAFINAL'].max()

        _participant[cat] = (pat_min,pat_max)
        
    _categories[pat] = _participant

In [39]:
data.index = list(range(len(data)))

In [40]:
data['TIMESTAMP'] = data['TIMESTAMP'].apply(lambda x: x.tz_localize(None))

In [41]:
new_feature = {}
for x in zip(data.index, data['PARTICIPANT'], data['TIMESTAMP']):
    if x[1] in _categories.keys():
        for cat in _categories[x[1]]:
            pat_min = _categories[x[1]][cat][0]
            pat_max = _categories[x[1]][cat][1]

            if (x[2] >= pat_min) & (x[2] <= pat_max):
                new_feature[x[0]] =  cat

In [42]:
new_feature = [new_feature[x] if x in new_feature else None for x in data.index]

In [43]:
data['CATEGORY'] = new_feature

Delete unused columns

In [44]:
data=data.drop(['TIMESTAMP','PARTICIPANT'], axis=1)

Delete the <b>"no_category"</b> data

In [45]:
new_data = data.loc[data['CATEGORY'] != "no_category"].copy()

<b>Method to Create the File for Each Category</b>

In [46]:
def create_category (new_data, write_data, category):
    new_df = pd.DataFrame(columns = ['ECG', 'EMG', 'EDA', 'CATEGORY'])
    df = new_data.loc[new_data['CATEGORY'] == category].copy()
    
    sampling_rate_reducer = 1000
    size = (len(df)-(sampling_rate_reducer*2))
    
    x = 0
    position = 0
    while (position < size) :
        new_df = new_df.append(df[position:position+sampling_rate_reducer])
        position = position+sampling_rate_reducer
        auxiliary = df[position:position+sampling_rate_reducer] 
        delta_ecg = auxiliary['ECG'].mean()
        delta_emg = auxiliary['EMG'].mean()
        delta_eda = auxiliary['EDA'].mean()
        delta_category = category
        position = position+sampling_rate_reducer
        delta = pd.DataFrame({'ECG' : [delta_ecg],'EMG' : [delta_emg],'EDA' : [delta_eda],'CATEGORY' : [delta_category]})
        new_df = new_df.append(delta)
    
    new_df.to_csv(write_data, index=None, header=True, sep=';', encoding='utf-8')

Create File for <b>"baseline"</b> Category

In [47]:
category = 'baseline'
file = write_data + category + file_type
create_category (new_data, file, category)

Create File for <b>"tsst"</b> Category

In [48]:
category = 'tsst'
file = write_data + category + file_type
create_category (new_data, file, category)

Create File for <b>"arithmetic"</b> Category

In [49]:
category = 'arithmetic'
file = write_data + category + file_type
create_category (new_data, file, category)

Create File for <b>"post_test_sensores_1"</b> Category

In [50]:
category = 'post_test_sensors_1'
file = write_data + category + file_type
create_category (new_data, file, category)

Create File for <b>"post_test_sensores_2"</b> Category

In [51]:
category = 'post_test_sensors_2'
file = write_data + category + file_type
create_category (new_data, file, category)